# SciTeX Pandas Module - Comprehensive Tutorial

This notebook demonstrates the complete functionality of the `scitex.pd` module for advanced pandas DataFrame operations, data manipulation, and statistical analysis utilities.

## Features Covered
* Universal DataFrame conversion with `force_df`
* Statistical column detection and organization
* Advanced data reshaping and melting
* Column and row movement operations
* Matrix to long format conversion
* Data type conversion utilities
* Warning suppression for clean workflows
* Complete data processing pipelines

## Table of Contents
1. [Universal DataFrame Conversion](#1-universal-dataframe-conversion)
2. [Statistical Column Detection](#2-statistical-column-detection)
3. [Data Organization Operations](#3-data-organization-operations)
4. [Advanced Data Reshaping](#4-advanced-data-reshaping)
5. [Matrix Format Conversions](#5-matrix-format-conversions)
6. [Data Type Utilities](#6-data-type-utilities)
7. [Advanced Operations](#7-advanced-operations)
8. [Real-World Applications](#8-real-world-applications)
9. [Complete Data Pipeline](#9-complete-data-pipeline)

In [ ]:
# Detect notebook name for output directory
import os
from pathlib import Path

# Get notebook name (for papermill compatibility)
notebook_name = "15_scitex_pd"
if 'PAPERMILL_NOTEBOOK_NAME' in os.environ:
    notebook_name = Path(os.environ['PAPERMILL_NOTEBOOK_NAME']).stem


## 1. Universal DataFrame Conversion

The `force_df` function converts any data type into a pandas DataFrame with intelligent handling of different input formats.

In [ ]:
# Import required libraries
import sys
sys.path.insert(0, '../src')
import scitex as stx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
import time

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)


### 1.1 Converting Various Data Types

In [ ]:
import pandas as pd
import numpy as np

# 1. Scalar value
scalar = 42
df_scalar = stx.pd.force_df(scalar)

# 2. List of values
list_data = [1, 2, 3, 4, 5]
df_list = stx.pd.force_df(list_data)

# 3. 1D NumPy array
array_1d = np.array([10, 20, 30, 40, 50])
df_array_1d = stx.pd.force_df(array_1d)

# 4. 2D NumPy array
array_2d = np.random.randn(4, 3)
df_array_2d = stx.pd.force_df(array_2d)

# 5. Dictionary with equal lengths
dict_equal = {
    'name': ['Alice', 'Bob', 'Charlie'],
    'age': [25, 30, 35],
    'city': ['New York', 'London', 'Tokyo']
}
df_dict_equal = stx.pd.force_df(dict_equal)

# 6. Dictionary with unequal lengths
dict_unequal = {
    'experiment_1': [23.5, 24.1, 23.8],
    'experiment_2': [22.9, 23.5, 24.0, 23.7, 23.9],
    'experiment_3': [24.2, 23.6]
}
df_dict_unequal = stx.pd.force_df(dict_unequal)

# 7. Pandas Series
series = pd.Series([100, 200, 300, 400], index=['Q1', 'Q2', 'Q3', 'Q4'], name='sales')
df_series = stx.pd.force_df(series)

### 1.2 Custom Filler Values for Unequal Lengths

In [ ]:
import pandas as pd
import numpy as np
# Demonstrate different filler strategies

# Sample data with unequal lengths
measurement_data = {
    'control_group': [98.6, 98.8, 99.1, 98.7],
    'treatment_a': [99.2, 99.5, 99.8, 100.1, 99.9, 100.3],
    'treatment_b': [97.8, 98.1]
}

for key, values in measurement_data.items():
        pass  # Processing key
# 1. Default filler (NaN)
df_nan = stx.pd.force_df(measurement_data)

# 2. Zero filler
df_zero = stx.pd.force_df(measurement_data, filler=0)

# 3. Mean filler
all_values = [v for vals in measurement_data.values() for v in vals]
mean_value = np.mean(all_values)
df_mean = stx.pd.force_df(measurement_data, filler=mean_value)

# 4. Custom string filler
df_missing = stx.pd.force_df(measurement_data, filler='MISSING')

# Compare statistics
fillers = ['NaN', 'Zero', 'Mean', 'String']
dataframes = [df_nan, df_zero, df_mean, df_missing]

for filler, df in zip(fillers, dataframes):
    if filler != 'String':  # Skip string data for numeric stats
    try:
        numeric_df = df.select_dtypes(include=[np.number])
        except:
            pass  # Handle exception
    else:        pass  # Fixed incomplete block


## 2. Statistical Column Detection

The `find_pval` function automatically detects p-value columns in DataFrames, useful for statistical analysis workflows.

In [ ]:
import pandas as pd

# Create a DataFrame with statistical results
statistical_results = {
    'feature': ['height', 'weight', 'age', 'blood_pressure', 'cholesterol'],
    'mean_control': [170.5, 68.2, 35.4, 120.5, 180.2],
    'mean_treatment': [172.1, 66.8, 35.6, 118.2, 175.8],
    'std_control': [8.5, 12.1, 8.9, 15.2, 25.4],
    'std_treatment': [9.1, 11.8, 9.2, 14.8, 23.9],
    'effect_size': [0.19, -0.11, 0.02, -0.15, -0.18],
    'p_value': [0.023, 0.156, 0.832, 0.041, 0.067],
    'pval_adjusted': [0.092, 0.312, 0.832, 0.164, 0.268],
    'confidence_interval_lower': [0.12, -0.18, -0.09, -0.22, -0.25],
    'confidence_interval_upper': [0.26, -0.04, 0.13, -0.08, -0.11],
    'significance': ['*', 'ns', 'ns', '*', 'ns'],
    'p-val-bonferroni': [0.115, 0.780, 1.000, 0.205, 0.335]
}

df_stats = pd.DataFrame(statistical_results)

# Find all p-value columns
pval_cols_all = stx.pd.find_pval(df_stats, multiple=True)

# Find only the first p-value column
pval_col_first = stx.pd.find_pval(df_stats, multiple=False)

# Extract and analyze p-values
for col in pval_cols_all:
    p_values = df_stats[col]
    significant = (p_values < 0.05).sum()
    total = len(p_values)

# Create significance summary
significance_summary = df_stats[['feature'] + pval_cols_all].copy()
for col in pval_cols_all:
    significance_summary[f'{col}_sig'] = significance_summary[col] < 0.05


## 3. Data Organization Operations

Functions for organizing DataFrames by moving columns and rows to specific positions.

### 3.1 Column Movement Operations

In [ ]:
import pandas as pd

# Create a sample DataFrame
data = {
    'subject_id': ['S001', 'S002', 'S003', 'S004', 'S005'],
    'age': [25, 30, 35, 40, 45],
    'gender': ['M', 'F', 'M', 'F', 'M'],
    'baseline_score': [85, 92, 78, 88, 91],
    'treatment_score': [88, 95, 82, 92, 94],
    'improvement': [3, 3, 4, 4, 3],
    'p_value': [0.023, 0.015, 0.048, 0.012, 0.031]
}

df_original = pd.DataFrame(data)

# 1. Move column to specific position
df_moved = stx.pd.mv(df_original, 'p_value', 2)

# 2. Move column to first position
df_first = stx.pd.mv_to_first(df_original, 'treatment_score')

# 3. Move column to last position
df_last = stx.pd.mv_to_last(df_original, 'subject_id')

# 4. Chain operations for complex reorganization
df_reorganized = (df_original
    .pipe(stx.pd.mv_to_first, 'p_value')  # P-value first
    .pipe(stx.pd.mv_to_first, 'subject_id')  # ID before p-value
    .pipe(stx.pd.mv_to_last, 'gender')  # Gender last
    )


# 5. Organize based on p-value detection
pval_col = stx.pd.find_pval(df_original)
if pval_col:
    df_pval_first = stx.pd.mv_to_first(df_original, pval_col)

### 3.2 Row Movement Operations

In [ ]:
import pandas as pd

# Create DataFrame with named index
experiment_data = {
    'measurement': [10.5, 20.3, 15.7, 25.1, 18.9],
    'error': [0.2, 0.4, 0.3, 0.5, 0.3],
    'category': ['baseline', 'treatment', 'control', 'recovery', 'followup']
}

df_rows = pd.DataFrame(experiment_data, 
    index=['exp_baseline', 'exp_treatment', 'exp_control',
    'exp_recovery', 'exp_followup'])


# 1. Move row to first position
df_row_first = stx.pd.mv_to_first(df_rows, 'exp_treatment', axis=0)

# 2. Move row to last position
df_row_last = stx.pd.mv_to_last(df_rows, 'exp_baseline', axis=0)

# 3. Logical ordering (treatment first, then control, then others)
desired_order = ['exp_treatment', 'exp_control', 'exp_baseline', 'exp_recovery', 'exp_followup']
df_logical = df_rows.reindex(desired_order)

# 4. Sort by measurement value and reorganize
df_sorted = df_rows.sort_values('measurement')
# Move highest value to first
highest_idx = df_sorted.index[-1]  # Last after sorting (highest)
df_highlight = stx.pd.mv_to_first(df_sorted, highest_idx, axis=0)


## 4. Advanced Data Reshaping

Specialized functions for reshaping data while preserving relationships.

### 4.1 Selective Column Melting

In [ ]:
import pandas as pd

# Create wide format longitudinal data
longitudinal_data = {
    'participant_id': ['P001', 'P002', 'P003', 'P004', 'P005'],
    'age': [25, 30, 35, 40, 45],
    'gender': ['M', 'F', 'M', 'F', 'M'],
    'baseline_anxiety': [12, 15, 18, 14, 16],
    'week_2_anxiety': [10, 13, 16, 12, 14],
    'week_4_anxiety': [8, 11, 14, 10, 12],
    'week_8_anxiety': [6, 9, 12, 8, 10],
    'baseline_depression': [8, 12, 15, 11, 13],
    'week_2_depression': [7, 10, 13, 9, 11],
    'week_4_depression': [6, 8, 11, 7, 9],
    'week_8_depression': [5, 6, 9, 5, 7]
}

df_wide = pd.DataFrame(longitudinal_data)

# 1. Melt only anxiety measurements
anxiety_cols = [col for col in df_wide.columns if 'anxiety' in col]
df_anxiety_long = stx.pd.melt_cols(df_wide, anxiety_cols)


# 2. Melt depression measurements with specific ID columns
depression_cols = [col for col in df_wide.columns if 'depression' in col]
df_depression_long = stx.pd.melt_cols(df_wide, depression_cols, 
    id_columns=['participant_id', 'age', 'gender'])


# 3. Melt all measurement columns (both anxiety and depression)
measurement_cols = anxiety_cols + depression_cols
df_all_long = stx.pd.melt_cols(df_wide, measurement_cols, 
    id_columns=['participant_id', 'age', 'gender'])


# 4. Parse time points and outcome types
df_parsed = df_all_long.copy()
df_parsed['time_point'] = df_parsed['variable'].str.extract(r'(baseline|week_\d+)')
df_parsed['outcome_type'] = df_parsed['variable'].str.extract(r'(anxiety|depression)')


# 5. Create summary statistics
summary_stats = (df_parsed.groupby(['time_point', 'outcome_type'])['value']
    .agg(['count', 'mean', 'std'])
    .round(2))


### 4.2 Complex Reshaping Scenarios

In [ ]:
import pandas as pd
# Multi-level experimental design

# Create complex experimental data
complex_data = {
    'lab_id': ['Lab_A', 'Lab_B', 'Lab_C'],
    'researcher': ['Dr. Smith', 'Dr. Jones', 'Dr. Brown'],
    # Condition 1 measurements
    'cond1_trial1': [45.2, 43.1, 46.8],
    'cond1_trial2': [44.8, 42.9, 47.1],
    'cond1_trial3': [45.5, 43.3, 46.9],
    # Condition 2 measurements
    'cond2_trial1': [52.1, 51.3, 53.2],
    'cond2_trial2': [51.8, 50.9, 52.8],
    'cond2_trial3': [52.3, 51.1, 53.1],
    # Quality metrics
    'equipment_calibrated': [True, True, False],
    'temperature': [22.1, 22.3, 21.8]
}

df_complex = pd.DataFrame(complex_data)

# Identify measurement columns
measurement_cols = [col for col in df_complex.columns if col.startswith('cond')]

# Melt measurements while preserving metadata
id_cols = ['lab_id', 'researcher', 'equipment_calibrated', 'temperature']
df_melted = stx.pd.melt_cols(df_complex, measurement_cols, id_columns=id_cols)


# Extract condition and trial information
df_extracted = df_melted.copy()
df_extracted['condition'] = df_extracted['variable'].str.extract(r'(cond\d+)')
df_extracted['trial'] = df_extracted['variable'].str.extract(r'(trial\d+)')


# Quality-filtered analysis
df_calibrated = df_extracted[df_extracted['equipment_calibrated'] == True]
quality_summary = (df_calibrated.groupby(['condition', 'trial'])['value']
    .agg(['mean', 'std', 'count'])
    .round(3))


# Compare calibrated vs non-calibrated
comparison = (df_extracted.groupby(['condition', 'equipment_calibrated'])['value']
    .mean()
    .unstack()
    .round(3))
comparison.columns = ['Not_Calibrated', 'Calibrated']
comparison['Difference'] = comparison['Calibrated'] - comparison['Not_Calibrated']


## 5. Matrix Format Conversions

Converting between wide matrix format and long x,y,z format for analysis and visualization.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create correlation matrix
np.random.seed(42)
n_variables = 6
variable_names = [f'var_{i+1}' for i in range(n_variables)]

# Generate correlated data
base_data = np.random.randn(200, n_variables)
# Add some correlations
base_data[:, 1] = 0.7 * base_data[:, 0] + 0.3 * base_data[:, 1]  # var_2 correlated with var_1
base_data[:, 3] = -0.5 * base_data[:, 2] + 0.5 * base_data[:, 3]  # var_4 anti-correlated with var_3

df_vars = pd.DataFrame(base_data, columns=variable_names)
corr_matrix = df_vars.corr()


# Convert to x,y,z (long) format
xyz_data = stx.pd.to_xyz(corr_matrix)

# Analyze correlation patterns

# Find strongest correlations (excluding diagonal)
off_diagonal = xyz_data[xyz_data['x'] != xyz_data['y']]
strongest_positive = off_diagonal.loc[off_diagonal['z'].idxmax()]
strongest_negative = off_diagonal.loc[off_diagonal['z'].idxmin()]


# Create distance matrix and convert
distance_matrix = 1 - np.abs(corr_matrix)  # Distance as 1 - |correlation|
distance_xyz = stx.pd.to_xyz(distance_matrix)



# Visualize both formats
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Correlation heatmap
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
    square=True, ax=axes[0,0], cbar_kws={'label': 'Correlation'})
axes[0,0].set_title('Correlation Matrix (Wide Format)')

# 2. Correlation scatter plot from long format
scatter1 = axes[0,1].scatter(xyz_data['x'], xyz_data['y'], c=xyz_data['z'], 
    cmap='RdBu_r', s=100, vmin=-1, vmax=1)
axes[0,1].set_xlabel('Variable 1')
axes[0,1].set_ylabel('Variable 2')
axes[0,1].set_title('Correlation Values (Long Format)')
axes[0,1].set_xticks(range(len(variable_names)))
axes[0,1].set_xticklabels(variable_names, rotation=45)
axes[0,1].set_yticks(range(len(variable_names)))
axes[0,1].set_yticklabels(variable_names)
axes[0,1].invert_yaxis()
plt.colorbar(scatter1, ax=axes[0,1], label='Correlation')

# 3. Distance heatmap
sns.heatmap(distance_matrix, annot=True, fmt='.2f', cmap='viridis',
    square=True, ax=axes[1,0], cbar_kws={'label': 'Distance'})
axes[1,0].set_title('Distance Matrix (1 - |correlation|)')

# 4. Correlation distribution
axes[1,1].hist(off_diagonal['z'], bins=20, alpha=0.7, edgecolor='black')
axes[1,1].axvline(0, color='red', linestyle='--', alpha=0.7, label='Zero correlation')
axes[1,1].set_xlabel('Correlation Value')
axes[1,1].set_ylabel('Frequency')
axes[1,1].set_title('Distribution of Correlations')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Data Type Utilities

Safe conversion of DataFrame columns to appropriate data types.

In [ ]:
import pandas as pd
import numpy as np

# Create DataFrame with mixed and problematic data types
mixed_data = {
    'integers_as_strings': ['1', '2', '3', '4', '5'],
    'floats_as_strings': ['1.5', '2.7', '3.14', '4.0', '5.5'],
    'mixed_with_errors': ['10', '20.5', 'invalid', '40', 'bad_data'],
    'percentages': ['10%', '25%', '50%', '75%', '100%'],
    'currency': ['$100', '$250.50', '$1,000', '$500', '$750.25'],
    'already_numeric': [1, 2, 3, 4, 5],
    'scientific_notation': ['1e2', '2.5e3', '1.2e-1', '5e0', '3.14e1'],
    'text_data': ['apple', 'banana', 'cherry', 'date', 'elderberry']
}

df_mixed = pd.DataFrame(mixed_data)

# Convert to numeric using SciTeX
df_numeric = stx.pd.to_numeric(df_mixed)

# Analyze conversion results
for col in df_mixed.columns:
    original_type = df_mixed[col].dtype
    converted_type = df_numeric[col].dtype
    
    if original_type != converted_type:
        # Count successful conversions
        if converted_type in ['int64', 'float64']:
            non_null = df_numeric[col].notna().sum()
            total = len(df_numeric[col])
            success_rate = non_null / total * 100
        else:
            pass  # Fixed incomplete block
    else:
        pass  # Fixed incomplete block

# Show which values couldn't be converted
for col in df_mixed.columns:
    if df_numeric[col].dtype in ['int64', 'float64']:
        failed_mask = df_numeric[col].isna() & df_mixed[col].notna()
        if failed_mask.any():
            failed_values = df_mixed.loc[failed_mask, col].tolist()

# Demonstrate safe numeric operations
numeric_cols = df_numeric.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    values = df_numeric[col].dropna()
    if len(values) > 0:
        # Condition met

# Create summary statistics
numeric_summary = df_numeric.select_dtypes(include=[np.number]).describe()

## 7. Advanced Operations

Additional utilities for DataFrame manipulation and analysis.

In [ ]:
import pandas as pd
import numpy as np

# Warning suppression utility

# Create DataFrame that might trigger SettingWithCopyWarning
df_original = pd.DataFrame({
    'A': range(10),
    'B': range(10, 20),
    'C': range(20, 30)
})

# This operation might normally trigger a warning
df_subset = df_original[df_original['A'] > 5]


# Without warning suppression (might show warning)
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    df_subset.loc[:, 'D'] = df_subset['A'] * df_subset['B']
    if w:
        # Condition met
    else:
        pass  # Fixed incomplete block

# With warning suppression
with stx.pd.ignore_SettingWithCopyWarning():
    df_subset.loc[:, 'E'] = df_subset['B'] + df_subset['C']


# 2. Column merging utility

# Create data with columns to merge
personal_data = {
    'first_name': ['John', 'Jane', 'Bob', 'Alice'],
    'middle_initial': ['A', 'B', None, 'C'],
    'last_name': ['Doe', 'Smith', 'Johnson', 'Brown'],
    'street': ['123 Main St', '456 Oak Ave', '789 Pine Rd', '321 Elm St'],
    'city': ['New York', 'London', 'Paris', 'Tokyo'],
    'country': ['USA', 'UK', 'France', 'Japan'],
    'age': [30, 25, 35, 28]
}

df_personal = pd.DataFrame(personal_data)

# Merge name columns (handling missing middle initial)
df_merged = df_personal.copy()

# Custom merge function for names with optional middle initial
def merge_names(row):
    parts = [row['first_name']]
    if pd.notna(row['middle_initial']):
        parts.append(row['middle_initial'] + '.')
    parts.append(row['last_name'])
    return ' '.join(parts)

df_merged['full_name'] = df_merged.apply(merge_names, axis=1)

# Merge address columns
df_merged['full_address'] = (df_merged['street'] + ', ' + 
    df_merged['city'] + ', ' +
    df_merged['country'])

# Keep only essential columns
df_final = df_merged[['full_name', 'full_address', 'age']]


# 3. Advanced slicing operations

# Create time series data
dates = pd.date_range('2024-01-01', periods=50, freq='D')
ts_data = {
    'date': dates,
    'temperature': 20 + 5 * np.sin(np.arange(50) * 2 * np.pi / 30) + np.random.randn(50),
    'humidity': 60 + 10 * np.cos(np.arange(50) * 2 * np.pi / 25) + np.random.randn(50) * 2,
    'pressure': 1013 + np.random.randn(50) * 5
}

df_ts = pd.DataFrame(ts_data)
df_ts.set_index('date', inplace=True)


# Slice by date range
start_date = '2024-01-15'
end_date = '2024-01-25'
date_slice = df_ts[start_date:end_date]


# Slice by condition
high_temp = df_ts[df_ts['temperature'] > df_ts['temperature'].mean()]

# Complex filtering
complex_filter = df_ts[
    (df_ts['temperature'] > 22) & 
    (df_ts['humidity'] < 65) & 
    (df_ts['pressure'] > 1010)
]

if len(complex_filter) > 0:
    f"humidity={complex_filter['humidity'].mean():.1f}, "
    f"pressure={complex_filter['pressure'].mean():.1f}")

## 8. Real-World Applications

Practical examples showing how the pandas utilities work in real data science scenarios.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Simulate clinical trial data with realistic challenges
np.random.seed(42)
n_patients = 100

# Generate patient data
patients = [f'PT_{i:03d}' for i in range(1, n_patients + 1)]
treatment_groups = np.random.choice(['Placebo', 'Drug_A', 'Drug_B'], n_patients)
ages = np.random.normal(55, 15, n_patients).astype(int)
ages = np.clip(ages, 18, 85)  # Realistic age range

# Create unequal measurement schedules (real-world missing data)
baseline_scores = np.random.normal(100, 15, n_patients)

# Week 4 - some patients drop out
week4_mask = np.random.random(n_patients) > 0.1  # 10% dropout
week4_scores = np.where(week4_mask, 
    baseline_scores + np.random.normal(-5, 8, n_patients),
    np.nan)

# Week 8 - more dropouts
week8_mask = week4_mask & (np.random.random(n_patients) > 0.15)  # Additional 15% dropout
week8_scores = np.where(week8_mask,
    baseline_scores + np.random.normal(-8, 10, n_patients),
    np.nan)

# Week 12 - final measurements (further dropouts)
week12_mask = week8_mask & (np.random.random(n_patients) > 0.2)  # Additional 20% dropout
week12_scores = np.where(week12_mask,
    baseline_scores + np.random.normal(-12, 12, n_patients),
    np.nan)

# Create the raw data dictionary (unequal lengths due to dropouts)
trial_data = {
    'patient_id': patients,
    'treatment_group': treatment_groups,
    'age': ages,
    'baseline_score': baseline_scores,
    'week_4_score': week4_scores,
    'week_8_score': week8_scores,
    'week_12_score': week12_scores
}

# Convert to DataFrame using force_df
df_trial = stx.pd.force_df(trial_data)


# Analyze dropout patterns
score_cols = ['baseline_score', 'week_4_score', 'week_8_score', 'week_12_score']
dropout_analysis = {}

for col in score_cols:
    available = df_trial[col].notna().sum()
    dropout_analysis[col] = {
    'available': available,
    'missing': len(df_trial) - available,
    'retention_rate': available / len(df_trial) * 100
    }

for timepoint, stats in dropout_analysis.items():
    f"{stats['missing']:2d} missing ({stats['retention_rate']:5.1f}% retention)")

# Reshape data for longitudinal analysis
df_long = stx.pd.melt_cols(df_trial, score_cols, 
    id_columns=['patient_id', 'treatment_group', 'age'])

# Extract timepoint information
df_long['timepoint'] = df_long['variable'].str.replace('_score', '')
df_long['weeks'] = df_long['timepoint'].map({
    'baseline': 0,
    'week_4': 4,
    'week_8': 8, 
    'week_12': 12
})


# Statistical analysis by treatment group
treatment_stats = (df_long.groupby(['treatment_group', 'timepoint'])['value']
    .agg(['count', 'mean', 'std'])
    .round(2))


# Calculate change from baseline
baseline_values = df_trial.set_index('patient_id')['baseline_score']

change_data = []
for _, row in df_trial.iterrows():
    patient_baseline = row['baseline_score']
    for col in ['week_4_score', 'week_8_score', 'week_12_score']:
        if pd.notna(row[col]):
            change_data.append({
            'patient_id': row['patient_id'],
            'treatment_group': row['treatment_group'],
            'timepoint': col.replace('_score', ''),
            'change_from_baseline': row[col] - patient_baseline
            })

df_changes = pd.DataFrame(change_data)

change_summary = (df_changes.groupby(['treatment_group', 'timepoint'])['change_from_baseline']
    .agg(['count', 'mean', 'std'])
    .round(2))

# Statistical testing
week12_changes = df_changes[df_changes['timepoint'] == 'week_12']

if len(week12_changes) > 0:
    groups = week12_changes['treatment_group'].unique()
    group_data = {}
    
    for group in groups:
        group_data[group] = week12_changes[week12_changes['treatment_group'] == group]['change_from_baseline']
    
    # Pairwise comparisons
    results = []
    for i, group1 in enumerate(groups):
        for group2 in groups[i+1:]:
            data1 = group_data[group1].dropna()
            data2 = group_data[group2].dropna()
            
            if len(data1) > 1 and len(data2) > 1:
                t_stat, p_val = stats.ttest_ind(data1, data2)
                results.append({
                'comparison': f'{group1}_vs_{group2}',
                'mean_diff': data1.mean() - data2.mean(),
                't_statistic': t_stat,
                'p_value': p_val
                })
    
    if results:
        df_stats_results = pd.DataFrame(results)
        
        # Move p-value column to front for emphasis
        df_stats_organized = stx.pd.mv_to_first(df_stats_results, 'p_value')
        
        
        # Find significant results
        significant = df_stats_organized[df_stats_organized['p_value'] < 0.05]
        
        if len(significant) > 0:
            # Condition met
else:
    pass  # Fixed incomplete block

# Visualize the results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Retention rates
timepoints = list(dropout_analysis.keys())
retention_rates = [dropout_analysis[tp]['retention_rate'] for tp in timepoints]
axes[0,0].plot(range(len(timepoints)), retention_rates, 'o-', linewidth=2, markersize=8)
axes[0,0].set_xticks(range(len(timepoints)))
axes[0,0].set_xticklabels([tp.replace('_', ' ').title() for tp in timepoints], rotation=45)
axes[0,0].set_ylabel('Retention Rate (%)')
axes[0,0].set_title('Patient Retention Over Time')
axes[0,0].grid(True, alpha=0.3)
axes[0,0].set_ylim(0, 105)

# 2. Treatment group scores over time
for group in df_trial['treatment_group'].unique():
    group_data = df_long[df_long['treatment_group'] == group]
    group_means = group_data.groupby('weeks')['value'].mean()
    axes[0,1].plot(group_means.index, group_means.values, 'o-', label=group, linewidth=2)

axes[0,1].set_xlabel('Weeks')
axes[0,1].set_ylabel('Score')
axes[0,1].set_title('Treatment Group Trajectories')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Change from baseline distribution
if len(df_changes) > 0:
    for group in df_changes['treatment_group'].unique():
        group_changes = df_changes[df_changes['treatment_group'] == group]['change_from_baseline']
        axes[1,0].hist(group_changes, alpha=0.6, label=group, bins=15, edgecolor='black')
    
    axes[1,0].set_xlabel('Change from Baseline')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].set_title('Distribution of Changes from Baseline')
    axes[1,0].legend()
    axes[1,0].axvline(0, color='red', linestyle='--', alpha=0.7)
    axes[1,0].grid(True, alpha=0.3)

# 4. Age distribution by treatment group
df_trial.boxplot(column='age', by='treatment_group', ax=axes[1,1])
axes[1,1].set_title('Age Distribution by Treatment Group')
axes[1,1].set_xlabel('Treatment Group')
axes[1,1].set_ylabel('Age (years)')
axes[1,1].get_figure().suptitle('')  # Remove automatic title

plt.tight_layout()
plt.show()


## 9. Complete Data Pipeline

A comprehensive example showing all pandas utilities working together in a complete data processing pipeline.

In [ ]:
import pandas as pd
import numpy as np

class AdvancedDataProcessor:
    """Complete data processing pipeline using SciTeX pandas utilities."""
    
    def __init__(self, name="DataProcessor"):
        self.name = name
        self.data = None
        self.processed = None
        self.steps_performed = []
        
    def load_data(self, data_source, filler=np.nan):
        """Load data from any source using force_df."""
        start_time = time.time()
        self.data = stx.pd.force_df(data_source, filler=filler)
        load_time = time.time() - start_time
        
        self.steps_performed.append({
        'step': 'load_data',
        'time': load_time,
        'shape': self.data.shape,
        'details': f'Loaded with filler={filler}'
        })
        
        return self
    
    def convert_numeric(self):
        """Convert columns to numeric where possible."""
        start_time = time.time()
        original_types = self.data.dtypes.to_dict()
        
        self.data = stx.pd.to_numeric(self.data)
        conversion_time = time.time() - start_time
        
        new_types = self.data.dtypes.to_dict()
        changed_cols = [col for col in original_types.keys() 
        if original_types[col] != new_types[col]]
        
        self.steps_performed.append({
        'step': 'convert_numeric',
        'time': conversion_time,
        'details': f'Converted {len(changed_cols)} columns to numeric'
        })
        
        return self
    
    def organize_statistics(self):
        """Find and organize statistical columns."""
        start_time = time.time()
        
        # Find p-value columns
        pval_cols = stx.pd.find_pval(self.data, multiple=True)
        
        if pval_cols:
            # Move first p-value column to front
            self.data = stx.pd.mv_to_first(self.data, pval_cols[0])
            
            # Move other statistical columns forward
            stat_keywords = ['effect', 'statistic', 'confidence', 'significant']
            stat_cols = [col for col in self.data.columns 
            if any(keyword in col.lower() for keyword in stat_keywords)]
            
            for col in stat_cols:
                if col != pval_cols[0]:  # Don't move p-value again
                try:
                    self.data = stx.pd.mv(self.data, col, len(pval_cols))
                    except:
                        pass  # Column might not exist or already moved
        
        organize_time = time.time() - start_time
        
        self.steps_performed.append({
            'step': 'organize_statistics',
            'time': organize_time,
            'details': f'Found {len(pval_cols)} p-value columns, organized statistical columns'
        })
        
        return self
    
    def reshape_longitudinal(self, value_vars, id_vars=None, time_var='time'):
        """Reshape data for longitudinal analysis."""
        start_time = time.time()
        
        if id_vars is None:
            # Auto-detect ID variables (non-numeric or explicitly named)
            id_candidates = []
            for col in self.data.columns:
                if (col.lower() in ['id', 'subject', 'patient', 'participant'] or 
                'id' in col.lower() or
                self.data[col].dtype == 'object'):
                id_candidates.append(col)
            
            # Remove value_vars from id_candidates
            id_vars = [col for col in id_candidates if col not in value_vars]
        
        self.processed = stx.pd.melt_cols(self.data, value_vars, id_columns=id_vars)
        
        # Extract time information if variable names contain time indicators
        if any(any(indicator in var.lower() for indicator in ['time', 'week', 'day', 'month', 'visit']) 
            for var in value_vars):
                # Process var
            self.processed[time_var] = (self.processed['variable']
                .str.extract(r'(\d+)')
                .astype(float))
        
        reshape_time = time.time() - start_time
        
        self.steps_performed.append({
            'step': 'reshape_longitudinal',
            'time': reshape_time,
            'details': f'Melted {len(value_vars)} columns with {len(id_vars)} ID variables'
        })
        
        return self
    
    def create_summary_matrix(self, index_col, columns_col, values_col, aggfunc='mean'):
        """Create summary matrix and convert to long format."""
        start_time = time.time()
        
        if self.processed is None:
            data_to_use = self.data
        else:
            data_to_use = self.processed
        
        # Create pivot table
        matrix = data_to_use.pivot_table(
            index=index_col,
            columns=columns_col,
            values=values_col,
            aggfunc=aggfunc
        )
        
        # Convert to xyz format
        xyz_data = stx.pd.to_xyz(matrix)
        
        matrix_time = time.time() - start_time
        
        self.steps_performed.append({
            'step': 'create_summary_matrix',
            'time': matrix_time,
            'details': f'Created {matrix.shape} matrix, converted to {xyz_data.shape} long format'
        })
        
        
        return {
            'matrix': matrix,
            'long_format': xyz_data
        }
    
    def get_results(self):
        """Get processed data and pipeline summary."""
        total_time = sum(step['time'] for step in self.steps_performed)
        
        return {
        'data': self.processed if self.processed is not None else self.data,
        'steps': self.steps_performed,
        'total_time': total_time,
        'pipeline_summary': self._create_summary()
        }
    
    def _create_summary(self):
        """Create pipeline execution summary."""
        return {
        'processor_name': self.name,
        'steps_completed': len(self.steps_performed),
        'total_time': sum(step['time'] for step in self.steps_performed),
        'final_shape': (self.processed if self.processed is not None else self.data).shape,
        'step_details': [
        f"{step['step']}: {step['time']:.3f}s - {step.get('details', '')}"
        for step in self.steps_performed
        ]
        }

# Example usage: Complex multi-source data processing

# Create complex, realistic dataset
np.random.seed(42)

# Simulate multi-site study data with various challenges
sites = ['Site_A', 'Site_B', 'Site_C']
n_subjects_per_site = [50, 45, 55]  # Unequal site sizes

all_data = []
subject_counter = 1

for site, n_subjects in zip(sites, n_subjects_per_site):
    for i in range(n_subjects):
        subject_id = f'{site}_S{subject_counter:03d}'
        
        # Baseline characteristics
        age = np.random.normal(50, 12)
        baseline_score = np.random.normal(100, 15)
        
        # Follow-up measurements with realistic dropout
        measurements = {
        'subject_id': subject_id,
        'site': site,
        'age': f'{age:.1f}',  # String to test conversion
        'baseline_measurement': baseline_score,
        }
        
        # Add follow-up measurements with increasing dropout
        for week in [4, 8, 12, 24]:
            dropout_prob = 0.05 + 0.02 * (week / 4)  # Increasing dropout over time
            if np.random.random() > dropout_prob:
                # Measurement with some improvement trend
                improvement = -week * 0.5 + np.random.normal(0, 5)
                measurements[f'week_{week}_measurement'] = baseline_score + improvement
        
        all_data.append(measurements)
        subject_counter += 1

# Add statistical results (simulated analysis outcomes)
comparison_data = {
    'comparison': ['Site_A_vs_Site_B', 'Site_A_vs_Site_C', 'Site_B_vs_Site_C'],
    'effect_size': [0.23, 0.15, -0.08],
    'p_value_unadjusted': [0.032, 0.156, 0.423],
    'p_value_bonferroni': [0.096, 0.468, 1.000],
    'confidence_interval_lower': [0.02, -0.05, -0.28],
    'confidence_interval_upper': [0.44, 0.35, 0.12],
    'significant_unadjusted': ['Yes', 'No', 'No']
}

# Process subject data
processor_subjects = AdvancedDataProcessor("Subject_Data_Processor")

subject_results = (processor_subjects
    .load_data(all_data, filler=np.nan)
    .convert_numeric()
    .get_results())

for detail in subject_results['pipeline_summary']['step_details']:
    # Process detail

# Process comparison statistics
processor_stats = AdvancedDataProcessor("Statistical_Results_Processor")

stats_results = (processor_stats
    .load_data(comparison_data)
    .convert_numeric()
    .organize_statistics()
    .get_results())

for detail in stats_results['pipeline_summary']['step_details']:
    # Process detail

# Longitudinal analysis
measurement_cols = [col for col in subject_results['data'].columns if 'measurement' in col]
id_cols = ['subject_id', 'site', 'age']

processor_longitudinal = AdvancedDataProcessor("Longitudinal_Processor")

longitudinal_results = (processor_longitudinal
    .load_data(subject_results['data'])
    .reshape_longitudinal(measurement_cols, id_cols)
    .get_results())

# Create site comparison matrix
matrix_results = processor_longitudinal.create_summary_matrix(
    index_col='site',
    columns_col='variable', 
    values_col='value'
)

for detail in longitudinal_results['pipeline_summary']['step_details']:
    # Process detail

# Final summary

# Show key results

# Show p-value organization
pval_col = stx.pd.find_pval(stats_results['data'])
if pval_col:
    significant_count = (stats_results['data'][pval_col] < 0.05).sum()


## Summary

This comprehensive tutorial has demonstrated the full capabilities of the SciTeX pandas module:

### Key Features Covered:
1. **Universal DataFrame Conversion** - `force_df` handles any data type with intelligent filling strategies
2. **Statistical Column Detection** - Automatic identification and organization of p-value columns
3. **Data Organization** - Flexible column and row movement for optimal data presentation
4. **Advanced Reshaping** - Selective melting and longitudinal data transformation
5. **Matrix Conversions** - Seamless conversion between wide and long formats for analysis
6. **Type Safety** - Robust numeric conversion with error handling
7. **Workflow Integration** - Complete data processing pipelines with performance tracking

### Best Practices Demonstrated:
- **Always use `force_df`** for consistent DataFrame creation from any data source
- **Organize statistical results** by moving p-value columns to prominent positions
- **Leverage selective melting** to preserve data relationships during reshaping
- **Convert formats strategically** between wide and long layouts for specific analyses
- **Handle missing data gracefully** with appropriate filler strategies
- **Chain operations efficiently** using method chaining for readable pipelines

### Performance Benefits:
- **Robust data loading** handles inconsistent input formats automatically
- **Intelligent type conversion** preserves data integrity while enabling numeric operations
- **Efficient reshaping** maintains data relationships during complex transformations
- **Streamlined workflows** reduce boilerplate code for common data science tasks

The SciTeX pandas module transforms complex data manipulation tasks into simple, reliable operations that scale from quick analyses to production data pipelines.